<a href="https://colab.research.google.com/github/FrancesBW/SPARK_development/blob/main/Collaboration_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup


In [1]:
import numpy as np
from astropy.io import fits
import astropy.table as pytabs
import matplotlib.pyplot as plt
import os
import pandas as pd

os.chdir('SPARK/SPARK/absorption/')
from absorption_nov import lbfgs_abs
os.chdir('../../../')

FileNotFoundError: ignored

In [6]:
os.listdir()

['.config', 'sample_data']

In [ ]:
path = ''
name = '3C225A'

cat = fits.getdata(path+"all_sponge_sources_table_tighter.fits")
data_s = pytabs.Table(cat)
idx_absline=np.where(data_s["NAMES"]==name)[0][0]
#we have to un-nan for some sources that we added nans to to make them fit in the table
nan_values=np.isnan(data_s[idx_absline]["VEL"])
numeric_values=[not i for i in nan_values]
v = data_s[idx_absline]["VEL"][numeric_values]
#correct velocities to model over
chiller_vel_idx=np.intersect1d(np.where(v>-100.),np.where(v<80.))

#initialise the reduced data
v = v[chiller_vel_idx]
Tb = data_s[idx_absline]["TB"][numeric_values][chiller_vel_idx]
tau = data_s[idx_absline]["TAU"][numeric_values][chiller_vel_idx]
rms_Tb=data_s[idx_absline]['SIG_TB'][numeric_values][chiller_vel_idx]
rms_tau=data_s[idx_absline]['SIG_TAU'][numeric_values][chiller_vel_idx]

#Channel spacing
dv = np.diff(v)[0]

#hdr
hdr=fits.Header()
hdr["CDELT3"] = dv
hdr["CRPIX3"] = 0
hdr["CRVAL3"] = v[0]*1.e3

#parameters                                                                                                                                                                                                                                                              
amp_fact_init = 2./3.
sig_init = 10.
iprint_init = -1
iprint = -1
maxiter_init = 15000
maxiter = 150000
corr_iter = 10

n_gauss = 9          #@param {type:"slider", min:1, max:25, step:1} 
fit_tol = 1.1          #@param {type:"slider", min:1, max:10, step:0.2}
mu_tol = 5./(dv/1000)  #@param {type:"slider", min:0, max:100, step:1}
sig_tol = 5./(dv/1000) #@param {type:"slider", min:0, max:100, step:1}
lb_amp = 0.
ub_amp = np.max(Tb)
lb_mu = 1
ub_mu = len(tau)
lb_sig= 1
ub_sig = len(tau)/2.

core = lbfgs_abs(Tb=Tb, tau=tau, hdr=hdr, rms_Tb=rms_Tb, rms_tau=rms_tau)

solution=core.run(n_gauss=n_gauss, 
                  lb_amp=lb_amp,
                  ub_amp=ub_amp,
                  lb_mu=lb_mu,
                  ub_mu=ub_mu,
                  lb_sig=lb_sig,
                  ub_sig=ub_sig, 
                  pcc_mu=mu_tol, 
                  pcc_sig=sig_tol, 
                  red_chi_sq_thres=fit_tol, 
                  amp_fact_init=amp_fact_init,
                  sig_init=sig_init,
                  maxiter=maxiter,
                  maxiter_init=maxiter_init, 
                  max_cor_iter=corr_iter, 
                  iprint=iprint,
                  iprint_init=iprint_init, 
                  init=0, 
                  prior=None)